In [129]:
#pkg imports
import http.client
import json
import csv
import placekey
import pandas as pd
from placekey.api import PlacekeyAPI

In [144]:
#section for configuring input files
apikey = open("apikey.txt", "r").read()
fileConfig1 = {
    "filepath": "propertySales",
    "parsetype": "str",
    "query_id": "sale_id",
    "street_address": "address",
    "city": "",
    #"region": "",
    "postal_code": "",
    "location_name": "",
    "latitude":"Y",
    "longitude":"X"
}
fileConfig2 = {
    "filepath": "blightData",
    "parsetype": "str",
    "query_id": "ticket_id",
    "street_address": "mailing_address",
    "city": "city",
    "region": "state",
    "postal_code": "zip_code",
    "location_name": "",
    "latitude":"Y",
    "longitude":"X"
}
apiParams = [
    "query_id",
    "latitude",
    "longitude",
    #"location_name",
    "street_address",
    "city",
    "postal_code",
    "region"
]
defaultParams1 = {
    "iso_country_code": "US",
    "region": "MI",
    "city": "Detroit"
}
defaultParams2 = {
    "iso_country_code": "US",
}
#output / api config
outputname = "output.csv"

pk_api = PlacekeyAPI(apikey)


arYrLpPOiRUUHJfSzWUWmAGuKdnllvcO


In [131]:
dataset1 = pd.read_csv(fileConfig1['filepath'], dtype= object)
dataset2 = pd.read_csv(fileConfig2['filepath'],  dtype= object)


In [145]:
#function for building api dataframe
def build_pk_df(df: pd.DataFrame, config: dict, defaults:dict):
    dfColumnMap = {}
    unusedParams = []
    for param in config.keys():
        if config[param] != "" and param in apiParams:
            dfColumnMap[config[param]] = param
        elif param in apiParams:
            unusedParams.append(param)
    api_df: pd.DataFrame = df.rename(columns=dfColumnMap)
    cols = list(dfColumnMap.values())
    api_df = api_df[cols]
    for uParam in unusedParams: 
        api_df[uParam] = None
    for p in defaults.keys():
        api_df[p] = defaults[p]
    api_df["latitude"] = pd.to_numeric(api_df["latitude"])
    api_df["longitude"] = pd.to_numeric(api_df["longitude"])
    return api_df

    

In [149]:
def getPlacekeys(api_df):
    api_json = json.loads(api_df.to_json(orient="records"))
    api_response = pk_api.lookup_placekeys(api_json)
    response_df = pd.read_json(json.dumps(api_response), dtype={'query_id': str})
    return response_df

In [148]:
def mergeData(placekey: pd.DataFrame, original: pd.DataFrame, query_id: str):
    merged_df = pd.merge(original, placekey, right_on= 'query_id', left_on=query_id, how='left')
    if 'error' in merged_df.columns:
        merged_df = merged_df.drop('error', axis=1)
    merged_df = merged_df.drop('query_id', axis=1)
    pk = merged_df.pop('placekey')
    merged_df.insert(0, 'placekey', pk)
    return merged_df

In [147]:
testData1 = dataset1.head(10000)
pk1_df = build_pk_df(testData1, fileConfig1, defaultParams1)
placekeys1 = getPlacekeys(pk1_df)

testData2 = dataset2.head(10000)
pk2_df = build_pk_df(testData2, fileConfig2, defaultParams2)
placekeys2 = getPlacekeys(pk2_df)

KeyboardInterrupt: 

In [137]:
merged1: pd.DataFrame = mergeData(placekeys1, testData1, "sale_id")
merged2: pd.DataFrame = mergeData(placekeys2, testData2, "ticket_id")


KeyError: 'query_id'

In [140]:
print(testData1.columns)
print(placekeys1.head())
print(pk1_df.head())

Index(['X', 'Y', 'sale_id', 'parcel_number', 'address', 'street_number',
       'street_prefix', 'street_name', 'unit_number', 'sale_number',
       'sale_date', 'sale_price', 'grantor', 'grantee', 'liber_page',
       'term_of_sale', 'sale_verification', 'sale_instrument',
       'property_transferred_percentage', 'property_class_code',
       'economic_condition_factor_neigh', 'ESRI_OID'],
      dtype='object')
Empty DataFrame
Columns: []
Index: []
  query_id     street_address   latitude  longitude     city postal_code  \
0  3749837  14006 GLASTONBURY  42.388139 -83.224446  Detroit        None   
1  4210008  19951 ASBURY PARK  42.438587 -83.210323  Detroit        None   
2  4210338  16881 ASBURY PARK  42.415134 -83.209319  Detroit        None   
3  3775822      10322 TIREMAN  42.351916 -83.161543  Detroit        None   
4  3663543       4158 LINCOLN  42.346942 -83.074014  Detroit        None   

  iso_country_code region  
0               US     MI  
1               US     MI  
2   

In [ ]:
def mergeDataSets(data1, data2):
    df = data1.merge(data2, how='inner',on='placekey')
    return df
final = mergeDataSets(merged1, merged2)
final.to_csv(outputname, index=False)

            placekey                X_x               Y_x  sale_id  \
0    22d@63v-457-3kf      -83.225165621  42.4351922770001  4137441   
1    225@63v-4b3-2hq       -82.98520245  42.4285619500001  4347123   
2    23n@63v-4j4-3kf  -83.1819290989999  42.4108766370001  4186399   
3    227@63v-48z-kzz      -82.922157219  42.4020406700001  4251432   
4    23m@63v-455-tgk      -83.154194327      42.421802229  4661279   
..               ...                ...               ...      ...   
141  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
142  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
143  229@63v-4j6-9mk      -83.144583703       42.33553671  4042808   
144  224@63v-4dx-9vf  -83.2867776149999  42.4389900690001  3787528   
145  22t@63v-4g2-fxq      -83.202503087      42.338036023  4102727   

    parcel_number           address street_number street_prefix street_name  \
0       22080084.   19516 GREENVIEW         19516           NaN   GREENVIEW   
1